# Туториал по двухэтапной модели

In [1]:
import datetime
import scipy
import dill
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import pyarrow.parquet as pq
from zipfile import ZipFile
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20)

from tqdm import tqdm
tqdm.pandas()  # <- added this line

In [2]:
pd.read_csv('../input/hack-the-cart/hist_data.csv')

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14
...,...,...,...,...,...,...,...,...
4529884,95619205,4620221347,2021-09-07 18:48:29,204351817,1.000,99.990,False,9.84
4529885,95619205,4620221347,2021-09-07 18:48:29,203523604,1.000,62.190,False,9.84
4529886,95619205,4620221347,2021-09-07 18:48:29,203566357,2.000,39.890,False,9.84
4529887,95619205,4620221347,2021-09-07 18:48:29,203566418,1.000,62.190,False,9.84


# Покупатели

In [3]:
hist_data = pd.read_csv('../input/hack-the-cart/hist_data.csv')
#hist_data.rename(columns={'id': 'user_id', 'cluster_id' : 'item_id'} , inplace=True)
hist_data.head()

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14


In [4]:
hist_data.dtypes

buyer_id               int64
pav_order_id           int64
created               object
item_id                int64
count                float64
price_sold           float64
flag_weight_goods       bool
weight               float64
dtype: object

In [5]:
hist_data['number_of_zero_buy'] = hist_data[hist_data['price_sold'] == 0].groupby('buyer_id')['pav_order_id'].transform('count')

In [6]:
hist_data['created'] = pd.to_datetime(hist_data['created'], infer_datetime_format=True)

In [7]:
hist_data.head()

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight,number_of_zero_buy
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14,NaN
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14,NaN
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14,NaN
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14,NaN
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14,NaN


In [8]:
hist_data.dtypes

buyer_id                       int64
pav_order_id                   int64
created               datetime64[ns]
item_id                        int64
count                        float64
price_sold                   float64
flag_weight_goods               bool
weight                       float64
number_of_zero_buy           float64
dtype: object

In [9]:
#Временные фичи

hist_data['month'] = hist_data['created'].dt.month
hist_data['day'] = hist_data['created'].dt.day
hist_data['dayofweek'] = hist_data['created'].dt.dayofweek
hist_data['hour'] = hist_data['created'].dt.hour

del hist_data['created']

In [10]:
hist_data

,buyer_id,pav_order_id,item_id,count,price_sold,flag_weight_goods,weight,number_of_zero_buy,month,day,dayofweek,hour
0,95203091,98506637863,202808329,1.000,79.990,False,11.14,NaN,7,1,3,0
1,95203091,98506637863,202953905,1.072,44.945,True,11.14,NaN,7,1,3,0
2,95203091,98506637863,203566452,1.000,69.990,False,11.14,NaN,7,1,3,0
3,95203091,98506637863,202820143,1.972,41.295,True,11.14,NaN,7,1,3,0
4,95203091,98506637863,204400422,1.000,269.990,False,11.14,NaN,7,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4529884,95619205,4620221347,204351817,1.000,99.990,False,9.84,NaN,9,7,1,18
4529885,95619205,4620221347,203523604,1.000,62.190,False,9.84,NaN,9,7,1,18
4529886,95619205,4620221347,203566357,2.000,39.890,False,9.84,NaN,9,7,1,18
4529887,95619205,4620221347,203566418,1.000,62.190,False,9.84,NaN,9,7,1,18


In [11]:
hist_data['total_price'] = hist_data['count'] * hist_data['price_sold']

In [12]:
hist_data['is_day_flag'] = hist_data['hour'].between(7,21)

In [13]:
hist_data['is_weekend_flag'] = hist_data['dayofweek'] >= 4

In [14]:
hist_data.head()

,buyer_id,pav_order_id,item_id,count,price_sold,flag_weight_goods,weight,number_of_zero_buy,month,day,dayofweek,hour,total_price,is_day_flag,is_weekend_flag
0,95203091,98506637863,202808329,1.000,79.990,False,11.14,NaN,7,1,3,0,79.99000,False,False
1,95203091,98506637863,202953905,1.072,44.945,True,11.14,NaN,7,1,3,0,48.18104,False,False
2,95203091,98506637863,203566452,1.000,69.990,False,11.14,NaN,7,1,3,0,69.99000,False,False
3,95203091,98506637863,202820143,1.972,41.295,True,11.14,NaN,7,1,3,0,81.43374,False,False
4,95203091,98506637863,204400422,1.000,269.990,False,11.14,NaN,7,1,3,0,269.99000,False,False


In [15]:
#hist_data.groupby('buyer_id', as_index=False).agg(avg_total_cost = ('total_price', 'mean'))
#hist_data.groupby('buyer_id', as_index=False).agg(std_total_cost = ('total_price', 'std'))
#hist_data.groupby('buyer_id', as_index=False).agg(the_most_expensive_purchase = ('total_price', 'max'))

#hist_data.groupby('buyer_id', as_index=False).agg(favorite_day = ('dayofweek', lambda x: scipy.stats.mode(x)[0]))
#hist_data.groupby('buyer_id', as_index=False).agg(is_day_most_flag = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]))
#hist_data.groupby('buyer_id', as_index=False).agg(is_weekend_most_flag = ('is_weekend_flag', lambda x: scipy.stats.mode(x)[1]))
#hist_data.groupby('buyer_id', as_index=False).agg(median_buy_hour = ('hour', 'median'))

#hist_data.groupby('buyer_id', as_index=False).agg(favorite_item_user = ('item_id', lambda x: lambda x: x.value_counts().index[1].reset_index())

# hist_datag = hist_data.groupby('buyer_id', as_index=False).agg(
#         avg_total_cost = ('total_price', 'mean'),
#         std_total_cost = ('total_price', 'std'),
#         the_most_expensive_purchase = ('total_price', 'max'),
#         favorite_day = ('dayofweek', lambda x: scipy.stats.mode(x)[0]),
#         favorite_item = ('item_id', lambda x: scipy.stats.mode(x)[0]),
#         is_day_most_flag = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]),
#         is_weekend_most_flag = ('is_weekend_flag', lambda x: scipy.stats.mode(x)[0]),
#         median_buy_hour = ('hour', 'median')
#         )

In [16]:
#hist_data.groupby('buyer_id')['item_id'].apply(lambda x: x.value_counts().index[1]).reset_index()

hist_data = hist_data.join(last_orders.set_index('buyer_id')['item_id'], on='buyer_id', rsuffix='rjid').rename(columns={'item_idrjid':'last_order'})

In [17]:
#val = hist_data.groupby('buyer_id')['item_id'].apply(lambda x: x.value_counts().index[1]).reset_index().set_index('buyer_id')

In [18]:
#val

In [19]:
#     hist_datag = hist_data.groupby('buyer_id', as_index=False).agg(
#         median_total_cost_user = ('total_price', 'median'),
#         mean_total_cost_user = ('total_price', 'mean'),
#         std_total_cost_user = ('total_price', 'std'),
#         median_price_sold_user = ('price_sold', 'median'),
#         mean_price_sold_hist_user = ('price_sold', 'mean'),
#         std_price_sold_user = ('price_sold', 'std'),
#         the_most_expensive_purchase_user = ('total_price', 'max'),
#         the_most_expensive_user = ('price_sold', 'max'),
#         favorite_day_user = ('dayofweek', lambda x: scipy.stats.mode(x)[0]),
#         favorite_item_user = ('item_id', lambda x: scipy.stats.mode(x)[0]),
#         is_day_most_flag_user = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]),
#         is_weekend_most_flag_user = ('is_weekend_flag', lambda x: scipy.stats.mode(x)[0]),
#         median_buy_hour_user = ('hour', 'median'),
#         buy_range_user = ('price_sold', lambda x: x.max() - x.min()),
#         number_of_unique_items_user = ('item_id',lambda x: x.nunique()),
#         number_of_unique_visits_user = ('pav_order_id',lambda x: x.nunique()),
#         item_in_orders_user = ('pav_order_id','count')
        
#         )

In [20]:
# val1 = hist_datag

In [21]:
# val1

In [22]:
# val2 = hist_data.groupby('buyer_id')['item_id'].apply(lambda x: x.value_counts().index[2]).reset_index()

In [23]:
# val1 = val1.merge(val2,on='buyer_id').rename(columns={'item_id':'second_value'})

In [24]:
# val1

In [25]:
def buyer_fs_generation(path):
    hist_data = pd.read_csv(path)
    
    hist_data['created'] = pd.to_datetime(hist_data['created'], infer_datetime_format=True)
    
    hist_data['month'] = hist_data['created'].dt.month
    hist_data['day'] = hist_data['created'].dt.day
    hist_data['dayofweek'] = hist_data['created'].dt.dayofweek
    hist_data['hour'] = hist_data['created'].dt.hour
    
    del hist_data['created']
    
    hist_data['total_price'] = hist_data['count'] * hist_data['price_sold']
    hist_data['is_day_flag'] = hist_data['hour'].between(7,21)
    hist_data['is_weekend_flag'] = hist_data['dayofweek'] >= 4
    
    val1 = hist_data.groupby('buyer_id')['item_id'].apply(lambda x: x.value_counts().index[1]).reset_index()
    val2 = hist_data.groupby('buyer_id')['item_id'].apply(lambda x: x.value_counts().index[2]).reset_index()
    val3 = hist_data.groupby('buyer_id')['item_id'].apply(lambda x: x.value_counts().index[3]).reset_index()
    val4 = hist_data.groupby('buyer_id')['item_id'].apply(lambda x: x.value_counts().index[4]).reset_index()
    
    hist_data['zero_price_sold_count'] = hist_data[hist_data.price_sold == 0].groupby(['item_id'])['price_sold'].transform('count')
    hist_data['zero_price_orders'] = hist_data[hist_data['zero_price_sold_count'] > 0].groupby(['buyer_id'])['pav_order_id'].transform('count')
    
    
    hist_datag = hist_data.groupby('buyer_id', as_index=False).agg(
        median_total_cost_user = ('total_price', 'median'),
        mean_total_cost_user = ('total_price', 'mean'),
        std_total_cost_user = ('total_price', 'std'),
        median_price_sold_user = ('price_sold', 'median'),
        mean_price_sold_hist_user = ('price_sold', 'mean'),
        std_price_sold_user = ('price_sold', 'std'),
        the_most_expensive_purchase_user = ('total_price', 'max'),
        the_most_expensive_user = ('price_sold', 'max'),
        favorite_day_user = ('dayofweek', lambda x: scipy.stats.mode(x)[0]),
        favorite_item_user = ('item_id', lambda x: scipy.stats.mode(x)[0]),
        is_day_most_flag_user = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]),
        is_weekend_most_flag_user = ('is_weekend_flag', lambda x: scipy.stats.mode(x)[0]),
        median_buy_hour_user = ('hour', 'median'),
        buy_range_user = ('price_sold', lambda x: x.max() - x.min()),
        number_of_unique_items_user = ('item_id',lambda x: x.nunique()),
        number_of_unique_visits_user = ('pav_order_id',lambda x: x.nunique()),
        item_in_orders_user = ('pav_order_id','count')
        
        )
    
    
    item_in_orders = hist_data.groupby(['item_id'])['pav_order_id'].transform('count')
    mean_per_week = hist_data.groupby(['buyer_id', 'dayofweek'])['pav_order_id'].count().groupby(['buyer_id']).mean().to_frame().rename(columns={'pav_order_id':'mean_order_per_week'})
    hist_datag = hist_datag.join(mean_per_week, on='buyer_id')
    
    hist_datag = hist_datag.merge(val1,on='buyer_id',how='left').rename(columns={'item_id':'second_favorite_item_user'})
    hist_datag = hist_datag.merge(val2,on='buyer_id',how='left').rename(columns={'item_id':'third_favorite_item_user'})
    hist_datag = hist_datag.merge(val3,on='buyer_id',how='left').rename(columns={'item_id':'fourth_favorite_item_user'})
    hist_datag = hist_datag.merge(val4,on='buyer_id',how='left').rename(columns={'item_id':'fifth_favorite_item_user'})
    
    hist_datag = hist_datag.merge(hist_data[['buyer_id','zero_price_orders']],how='left').fillna(0)
    hist_datag.drop_duplicates(subset=['buyer_id'],inplace=True)
    
    hist_datag.to_csv('user_features.csv' ,index=False)
    return hist_datag

In [26]:
data = buyer_fs_generation('../input/hack-the-cart/hist_data.csv')

In [27]:
data

,buyer_id,median_total_cost_user,mean_total_cost_user,std_total_cost_user,median_price_sold_user,mean_price_sold_hist_user,std_price_sold_user,the_most_expensive_purchase_user,the_most_expensive_user,favorite_day_user,favorite_item_user,is_day_most_flag_user,is_weekend_most_flag_user,median_buy_hour_user,buy_range_user,number_of_unique_items_user,number_of_unique_visits_user,item_in_orders_user,mean_order_per_week,second_favorite_item_user,third_favorite_item_user,fourth_favorite_item_user,fifth_favorite_item_user,zero_price_orders
0,94578442,133.99000,152.759769,100.008995,94.9900,128.370378,109.095590,393.9900,419.994,6,202801709,True,True,17.0,392.934,31,3,37,12.333333,202979560,203438161,203529348,203457496,0.0
37,94578475,96.95000,142.550292,159.480331,71.4175,118.356974,145.519041,848.9900,848.990,5,202791192,True,True,15.0,820.495,38,2,38,19.000000,203083929,203011581,203566429,202880254,0.0
75,94578533,73.99000,104.827176,100.222633,53.9900,78.948548,75.936276,599.9800,337.990,4,202807066,True,False,12.0,335.500,54,4,62,20.666667,202864758,202915968,203586026,202872237,0.0
137,94578595,127.49000,216.666559,296.156788,99.9900,127.013814,78.101078,1803.7320,363.987,1,202799674,True,False,12.0,350.042,36,3,43,14.333333,202807477,202799674,202801849,202806977,0.0
180,94578656,69.94000,93.898496,70.958066,64.1400,84.125750,63.949865,335.4900,335.490,1,202838754,True,False,16.0,321.645,63,6,80,16.000000,202872237,202820148,202910396,202807411,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4529785,96395136,120.49180,170.236085,172.101776,99.8900,184.761333,180.757894,701.5900,701.590,0,202793061,True,False,12.0,666.800,15,1,15,15.000000,205757948,202818687,203576721,202820148,0.0
4529800,96397556,40.92806,72.030996,113.797652,38.0600,52.020426,41.847103,767.5884,181.280,0,202806879,True,False,18.0,172.220,47,1,47,47.000000,205768224,203043384,203383963,203040884,0.0
4529847,96399270,63.00500,82.127500,48.036046,63.0050,80.606875,49.528043,194.5700,194.570,0,202793061,False,False,22.0,170.240,16,1,16,16.000000,202824511,204070431,212519969,202820468,0.0
4529863,96399769,103.44000,121.454403,73.897419,93.1400,112.547714,76.417711,288.9900,288.990,1,202793169,False,False,3.0,269.100,14,1,14,14.000000,203469410,202963771,203410558,203382183,0.0


In [28]:
data['buyer_id'].drop_duplicates()

0          94578442
37         94578475
75         94578533
137        94578595
180        94578656
             ...   
4529785    96395136
4529800    96397556
4529847    96399270
4529863    96399769
4529877    96400990
Name: buyer_id, Length: 63925, dtype: int64

In [29]:
data['buyer_id'].drop_duplicates()

0          94578442
37         94578475
75         94578533
137        94578595
180        94578656
             ...   
4529785    96395136
4529800    96397556
4529847    96399270
4529863    96399769
4529877    96400990
Name: buyer_id, Length: 63925, dtype: int64

# Айтемы

In [30]:
items = pd.read_csv('../input/hack-the-cart/hist_data.csv')
#hist_data.rename(columns={'id': 'user_id', 'cluster_id' : 'item_id'} , inplace=True)
items.head()

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14


In [31]:
items['created'] = pd.to_datetime(items['created'], infer_datetime_format=True)

items['month'] = items['created'].dt.month
items['day'] = items['created'].dt.day
items['dayofweek'] = items['created'].dt.dayofweek
items['hour'] = items['created'].dt.hour

items['is_day_flag'] = items['hour'].between(7,21)

del items['created']

In [32]:
# unique_orders = items['pav_order_id'].nunique()

#items.groupby('item_id', as_index=False).agg(favorite_day = ('dayofweek', lambda x: scipy.stats.mode(x)[0]))
# #items.groupby('item_id', as_index=False).agg(is_day_most_flag = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]))

# #items.groupby('item_id', as_index=False).agg(avg_amout = ('count', 'mean'))
# #items.groupby('item_id', as_index=False).agg(std_amout = ('count', 'std')) дает дофига NaN
# #items.groupby('item_id', as_index=False).agg(max_amout = ('count', 'max'))

# items.groupby('item_id', as_index=False).agg(percentage_in_orders = ('pav_order_id', lambda x: x.count()/unique_orders))

In [33]:
def item_fs_generation(path):
    hist_data = pd.read_csv(path)
    
    hist_data['created'] = pd.to_datetime(hist_data['created'], infer_datetime_format=True)
    
    hist_data['month'] = hist_data['created'].dt.month
    hist_data['day'] = hist_data['created'].dt.day
    hist_data['dayofweek'] = hist_data['created'].dt.dayofweek
    hist_data['hour'] = hist_data['created'].dt.hour
    
    del hist_data['created']
    
    hist_data['is_day_flag'] = hist_data['hour'].between(7,21)
    hist_data['is_weekend_flag'] = hist_data['dayofweek'] >= 4
    hist_data['total_price'] = hist_data['count'] * hist_data['price_sold']

    
    hist_data['price_s_min'] = hist_data.groupby(['item_id'])['price_sold'].transform('min')
    hist_data['price_s_max'] = hist_data.groupby(['item_id'])['price_sold'].transform('max')
    hist_data['price_s_mean'] = hist_data.groupby(['item_id'])['price_sold'].transform('mean')
    hist_data['price_s_std'] = hist_data.groupby(['item_id'])['price_sold'].transform('std')

    hist_data['price_s_max_min'] = hist_data['price_s_max'] - hist_data['price_s_min']
    
    hist_data['mean_count'] = hist_data.groupby(['item_id'])['count'].transform('mean')
    
    hist_data['zero_price_sold_count'] = hist_data[hist_data.price_sold == 0].groupby(['item_id'])['price_sold'].transform('count')
    
    
    
    
    unique_orders = hist_data['pav_order_id'].nunique()
    mean_price = hist_data['price_sold'].mean()
    
    hist_datag = hist_data.groupby('item_id', as_index=False).agg(
        favorite_day_items = ('dayofweek', lambda x: scipy.stats.mode(x)[0]),
        is_day_most_flag_items = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]),
        std_total_cost_user = ('total_price', 'std'),
        median_price_sold_items = ('price_sold', 'median'),
        mean_price_sold_hist_items = ('price_sold', 'mean'),
        std_price_sold_items = ('price_sold', 'std'),
        median_amout_items = ('count', 'median'),
        mean_amout_items = ('count', 'median'),
        max_amout_items = ('count', 'max'),
        range_amout_items = ('count', lambda x: x.max() - x.min()),
        percentage_in_orders_items = ('pav_order_id', lambda x: x.count() / unique_orders),
        is_weekend_most_flag_items = ('is_weekend_flag', lambda x: scipy.stats.mode(x)[0]),
        item_cost_defference_from_median = ('price_sold', lambda x: x.median() - mean_price),
        median_buy_hour_items = ('hour', 'median'),
        price_s_mean_items = ('price_sold','mean'),
        price_s_median_items = ('price_sold','median'),
        item_bought_count_items = ('item_id','count')
        
        
    )
    
    hist_datag = hist_datag.join(hist_data['price_s_max_min'])
    hist_datag = hist_datag.join(hist_data['mean_count'])
    hist_datag = hist_datag.merge(hist_data[['item_id','zero_price_sold_count']],how='left').fillna(0)
    
    hist_datag.drop_duplicates(subset=['item_id'],inplace=True)
    
    hist_datag.to_csv('item_features.csv', index=False)
    return hist_datag

In [34]:
item_fs_generation('../input/hack-the-cart/hist_data.csv')

,item_id,favorite_day_items,is_day_most_flag_items,std_total_cost_user,median_price_sold_items,mean_price_sold_hist_items,std_price_sold_items,median_amout_items,mean_amout_items,max_amout_items,range_amout_items,percentage_in_orders_items,is_weekend_most_flag_items,item_cost_defference_from_median,median_buy_hour_items,price_s_mean_items,price_s_median_items,item_bought_count_items,price_s_max_min,mean_count,zero_price_sold_count
0,202782406,6,True,26.963275,14.99,12.545783,3.502262,2.000,2.000,20.000,19.0,0.000359,False,-89.653718,13.0,12.545783,14.99,83,48.70,1.272340,0.0
83,202782407,2,True,0.000000,149.99,149.990000,0.000000,1.000,1.000,1.000,0.0,0.000004,False,45.346282,16.0,149.990000,149.99,1,451.53,0.840900,0.0
84,202782412,4,True,299.000000,299.00,299.000000,0.000000,2.000,2.000,3.000,2.0,0.000013,True,194.356282,11.0,299.000000,299.00,3,69.99,2.140085,0.0
87,202782414,5,True,34.013023,89.99,89.990000,0.000000,1.000,1.000,2.000,1.0,0.000030,False,-14.653718,15.0,89.990000,89.99,7,499.95,1.025983,0.0
94,202782415,3,True,46.613554,104.61,109.438889,29.939829,1.000,1.000,3.000,2.0,0.000078,False,-0.033718,12.0,109.438889,104.61,18,190.52,1.039855,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4529884,218175579,0,False,0.000000,106.99,106.990000,0.000000,1.000,1.000,1.000,0.0,0.000004,False,2.346282,23.0,106.990000,106.99,1,49.05,1.209302,0.0
4529885,218199321,0,True,0.000000,199.99,199.990000,0.000000,1.000,1.000,1.000,0.0,0.000004,False,95.346282,18.0,199.990000,199.99,1,80.35,1.176277,0.0
4529886,218217955,5,True,0.000000,43.99,43.990000,0.000000,1.000,1.000,1.000,0.0,0.000004,True,-60.653718,10.0,43.990000,43.99,1,519.96,1.215977,0.0
4529887,218217991,6,True,0.000000,449.00,449.000000,0.000000,0.055,0.055,0.055,0.0,0.000004,True,344.356282,16.0,449.000000,449.00,1,499.95,1.025983,0.0
